## 支持向量机

SMO（序列最小优化），是SVM中最流行的一种实现  
优点：泛化错误率低，计算开销不大，结果易解释。  
缺点：对参数调节和核函数的选择敏感，原始分类器不加修改仅适用于处理二类问题。  
使用数据类型：数值型和标称型数据。  

#### 分隔超平面：  
&ensp; &ensp;将数据集分隔开来的对象；数据点在二维平面上，分隔超平面就是一条直线，数据点在三维空间中，分隔超平面就是一个平面；...数据点在一个1024维空间中，就需要一个1023维的某对象对数据进行分隔；用来分隔的对象就叫做超平面，也是分类的决策边界   
#### 间隔：  
&ensp; &ensp;点到分割面的距离  
#### SVM:  
&ensp; &ensp;找到离分隔超平面最近的点，并确保他们离分隔超平面的距离尽可能的远   
&ensp; &ensp;SVM本身是一个二类分类器，对多类问题应用SVM需要对代码做一些修改。  
#### 支持向量：
&ensp;&ensp;离超平面最近的那些点
#### 点到分隔超平面的距离：
$$|\frac{w^Tx+b}{||w||}|$$
#### SVM分类器求解的优化问题：
输入数据给分类器会输出一个类别标签，类别标签选择-1，+1.是因为+1，-1仅仅相差一个符号，方便数学上的处理，而且可以通过一个统一的公式表示间隔或者数据点到分隔超平面的距离，而不用担心数据到底属于-1类还是+1类。（label*($w^Tx+b$),此表达式始终是一个正数)  

$$y =\mathop{\min}_{n}(label\cdot (w^Tx+b))\cdot \frac{1}{\|w\|} $$
$$\mathop{\arg\max}_{w,b} y $$



In [4]:
import numpy as np

#加载数据集
def loadDataSet(fileName):
    dataMat=[]
    labelMat=[]
    fr=open(fileName)
    for line in fr.readlines():
        lineArr=line.strip().split('\t')
        dataMat.append([float(lineArr[0]),float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

def selectJrand(i,m):
    j=i
    while j==i:
        j=int(np.random.uniform(0,m))
    return j

#调整alpha
def clipAlpha(aj,H,L):
    if aj>H:
        aj=H
    if L>aj:
        aj=L
    return aj



In [5]:
dataArr,labelArr=loadDataSet('testSet.txt')
print(labelArr)

[-1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]


In [6]:
#简化版SMO算法
def smoSimple(dataMatIn,classLabels,C,toler,maxIter):  #参数分别为：数据集，类别标签，常数C，容错率，退出前最大的循环次数
    dataMatrix=np.mat(dataMatIn)
    labelMat=np.mat(classLabels).transpose() #类别标签转置，成为一个列向量
    b=0
    m,n=np.shape(dataMatrix)
    alphas=np.mat(np.zeros((m,1)))
    iter=0   #没有任何alpha改变下遍历数据集的次数
    while (iter<maxIter):
        alphaPairsChanged=0 #记录alpha是否已经进行优化
        for i in range(m):
            fXi=float(np.multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T))+b   #g(x)----预测的类别
            Ei=fXi-float(labelMat[i])  #函数g(x)对输入的预测值与真实值的差距，误差很大就会对该数据实例所对应的的alpha进行优化
            if ((labelMat[i]*Ei<-toler) and (alphas[i]<C)) or ((labelMat[i]*Ei>toler) and (alphas[i]>0)):
                j=selectJrand(i,m)  #选择一个与i不同的数
                fXj=float(np.multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T))+b
                Ej=fXj-float(labelMat[j])
                alphaIold=alphas[i].copy()
                alphaJold=alphas[j].copy()
                if (labelMat[i]!=labelMat[j]):
                    L=max(0,alphas[j]-alphas[i])
                    H=min(C,C+alphas[j]-alphas[i])
                else:
                    L=max(0,alphas[j]+alphas[i]-C)
                    H=min(C,alphas[j]+alphas[i])
                if L==H:
                    print("L==H")
                    continue
                eta=2.0*dataMatrix[i,:]*dataMatrix[j,:].T-dataMatrix[i,:]*dataMatrix[i,:].T-dataMatrix[j,:]*dataMatrix[j,:].T #最优修改量
                if eta>=0:  #详细的介绍和实现得看Platt原文
                    print("eta>=0")
                    continue
                alphas[j]-=labelMat[j]*(Ei-Ej)/eta
                alphas[j]=clipAlpha(alphas[j],H,L)
                if (abs(alphas[j]-alphaJold)<0.0001): #alpha[j]是否有轻微的变化
                    print("j not moving enough")
                    continue
                alphas[i]+=labelMat[j]*labelMat[i]*(alphaJold-alphas[j]) #alpha[i],alpha[j]改变的大小一样，方向不一样？
                b1=b-Ei-labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T- \
                    labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2=b-Ej-labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T- \
                    labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                if (0<alphas[i]) and (C>alphas[i]):
                    b=b1
                elif (0<alphas[j]) and (C>alphas[j]):
                    b=b2
                else:
                    b=(b1+b2)/2.0
                alphaPairsChanged+=1
                print("iter: %d i:%d,pairs changed %d" % (iter,i,alphaPairsChanged))
        if (alphaPairsChanged==0):
            iter +=1
        else:
            iter=0
        print("iteration number:%d" % iter)
    return b,alphas
    
    
                    
                        

In [7]:
b,alphas=smoSimple(dataArr,labelArr,0.6,0.001,40)

iter: 0 i:0,pairs changed 1
L==H
L==H
iter: 0 i:5,pairs changed 2
L==H
L==H
L==H
iter: 0 i:18,pairs changed 3
L==H
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
L==H
L==H
iter: 0 i:54,pairs changed 4
L==H
L==H
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:0
iter: 0 i:0,pairs changed 1
j not moving enough
j not moving enough
j not moving enough
L==H
iter: 0 i:18,pairs changed 2
L==H
j not moving enough
j not moving enough
L==H
j not moving enough
iter: 0 i:54,pairs changed 3
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
iteration number:0
j not moving enough
L==H
j not moving enough
L==H
L==H
j not moving enough
L==H
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
j not moving enough
L==H
j not moving enough
L==H
L==H
j not moving enough
j not moving enough
j not moving enough
iter: 0 i:96,pairs changed 1
iteration number:0

j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iter: 0 i:54,pairs changed 1
j not moving enough
j not moving enough
j not moving enough
iteration number:0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:3
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:4
j not moving eno

j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iter: 1 i:54,pairs changed 1
j not moving enough
j not moving enough
iteration number:0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:1
j not moving enough
iter: 1 i:29,pairs changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:0
j not moving enough
j not moving enough
j not moving enough
iter: 0 i:52,pairs changed 1
j not moving enough
j not moving enough
iteration number:0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:2


j not moving enough
j not moving enough
iteration number:0
j not moving enough
j not moving enough
j not moving enough
iteration number:1
j not moving enough
j not moving enough
j not moving enough
iteration number:2
j not moving enough
j not moving enough
j not moving enough
iteration number:3
j not moving enough
j not moving enough
j not moving enough
iteration number:4
j not moving enough
j not moving enough
j not moving enough
iteration number:5
j not moving enough
j not moving enough
j not moving enough
iteration number:6
j not moving enough
j not moving enough
j not moving enough
iteration number:7
j not moving enough
j not moving enough
j not moving enough
iteration number:8
j not moving enough
j not moving enough
j not moving enough
iteration number:9
j not moving enough
j not moving enough
j not moving enough
iteration number:10
j not moving enough
j not moving enough
j not moving enough
iteration number:11
j not moving enough
j not moving enough
j not moving enough
iteration 

In [8]:
print(b,alphas[alphas>0])

[[-3.78290463]] [[0.11472812 0.20923087 0.03868631 0.03819429 0.32445102]]


In [9]:
np.shape(alphas[alphas>0])

(1, 5)

In [10]:
#输出支持向量
for i in range(100):
    if alphas[i]>0.0:
        print(dataArr[i],labelArr[i])

[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[2.893743, -1.643468] -1.0
[5.286862, -2.358286] 1.0
[6.080573, 0.418886] 1.0


在几百个点组成的小规模数据集上，简化版SMO算法的运行没有什么问题，但是在更大的数据集上的运行速度就会变慢。  
接下来看完整的Platt SMO算法，实现alpha的更改和代数运算的优化环节一模一样，在优化过程中唯一的不同就是选择alpha的方式。  
完整版的Platt SMO算法应用了一些能够提速的启发方法。

In [37]:
#完整版PlattSMO的支持函数
import numpy as np
# class optStruct:
#     def __init__(self,dataMatIn,classLabels,C,toler):
#         self.X=dataMatIn
#         self.labelMat=classLabels
#         self.C=C
#         self.tol=toler
#         self.m=np.shape(dataMatIn)[0]
#         self.alphas=np.mat(np.zeros((self.m,1)))
#         self.b=0
#         self.eCache=np.mat(np.zeros((self.m,2)))

def calcEk(oS,k):
    fXk=float(np.multiply(oS.alphas,oS.labelMat).T*(oS.X*oS.X[k,:].T))+oS.b
    Ek=fXk-float(oS.labelMat[k])
    return Ek

def selectJ(i,oS,Ei):
    maxK=-1
    maxDeltaE=0
    Ej=0
    oS.eCache[i]=[1,Ei]
    validEcacheList=np.nonzero(oS.eCache[:,0].A)[0]
    if (len(validEcacheList))>1:
        for k in validEcacheList:
            if k==i:
                continue
            Ek=calcEk(oS,k)
            deltaE=abs(Ei-Ek)
            if (deltaE>maxDeltaE):
                maxK=k
                maxDeltaE=deltaE
                Ej=Ek
        return maxK,Ej
    else:
        j=selectJrand(i,oS.m)
        Ej=calcEk(oS,j)
    return j,Ej

def updateEk(oS,k):
    Ek=calcEk(oS,k)
    oS.eCache[k]=[1,Ek]


In [2]:
#完整版PlattSMO算法中的优化例程
def innerL(i,oS):
    Ei=calcEk(oS,i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i]>0)):
        j,Ej=selectJ(i,oS,Ei)
        alphaIold=oS.alphas[i].copy()
        alphaJold=oS.alphas[j].copy()
        if (oS.labelMat[i] != oS.labelMat[j]):
            L=max(0,oS.alphas[j]-oS.alphas[i])
            H=min(oS.C,oS.C+oS.alphas[j]-oS.alphas[i])
        else:
            L=max(0,oS.alphas[j]+oS.alphas[i]-oS.C)
            H=min(oS.C,oS.alphas[j]+oS.alphas[i])
        if L==H:
            print("L==H")
            return 0
        eta=2.0*oS.X[i,:]*oS.X[j,:].T-oS.X[i,:]*oS.X[i,:].T-oS.X[j,:]*oS.X[j,:].T
        if eta>=0:
            print("eta>=0")
            return 0
        oS.alphas[j]-=oS.labelMat[j]*(Ei-Ej)/eta
        oS.alphas[j]=clipAlpha(oS.alphas[j],H,L)
        updateEk(oS,j)
        if (abs(oS.alphas[j]-alphaJold)<0.00001):
            print("j not moving enough")
            return 0
        oS.alphas[i]+=oS.labelMat[j]*oS.labelMat[i]*(alphaJold-oS.alphas[j])
        updateEk(oS,i)
        b1=oS.b-Ei-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS.X[i,:].T-\
            oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[i,:]*oS.X[j,:].T
        b2=oS.b-Ej-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS.X[j,:].T-\
            oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[j,:]*oS.X[j,:].T
        if (0<oS.alphas[i]) and (oS.C>oS.alphas[j]):
            oS.b=b1
        elif (0<oS.alphas[j]) and (oS.C>oS.alphas[j]):
            oS.b=b2
        else:
            oS.b=(b1+b2)/2.0
        return 1
    else:
        return 0

In [45]:
#完整版PlattSMO的外循环代码
def smoP(dataMatIn,classLabels,C,toler,maxIter,kTup=('lin',0)):
    oS=optStruct(np.mat(dataMatIn),np.mat(classLabels).transpose(),C,toler,kTup)
    iter=0
    entireSet=True
    alphaPairsChanged=0
    while (iter<maxIter) and ((alphaPairsChanged>0) or (entireSet)):
        alphaPairsChanged=0
        if entireSet:
            for i in range(oS.m):
                alphaPairsChanged+=innerL(i,oS)
                print("fullSet,iter: %d i:%d,pairs changed %d" % (iter,i ,alphaPairsChanged))
            iter += 1
        else:
            nonBoundIs=np.nonzero((oS.alphas.A>0)*(oS.alphas.A<C))[0]
            for i in nonBoundIs:
                alphaPairsChanged+=innerL(i,oS)
                print("non-bound,iter:%d i %d,pairs changed %d" % (iter,i,alphaPairsChanged))
            iter +=1
        if entireSet:
            entireSet=False
        elif (alphaPairsChanged==0):
            entireSet=True
        print("iteration number:%d" % iter)
    return oS.b,oS.alphas
    
                    

In [49]:
dataArr,labelArr=loadDataSet('testSet.txt')
b,alphas=smoP(dataArr,labelArr,0.6,0.001,40)

fullSet,iter: 0 i:0,pairs changed 1
fullSet,iter: 0 i:1,pairs changed 1
fullSet,iter: 0 i:2,pairs changed 1
fullSet,iter: 0 i:3,pairs changed 2
fullSet,iter: 0 i:4,pairs changed 3
fullSet,iter: 0 i:5,pairs changed 3
fullSet,iter: 0 i:6,pairs changed 3
fullSet,iter: 0 i:7,pairs changed 3
j not moving enough
fullSet,iter: 0 i:8,pairs changed 3
fullSet,iter: 0 i:9,pairs changed 3
fullSet,iter: 0 i:10,pairs changed 4
fullSet,iter: 0 i:11,pairs changed 4
fullSet,iter: 0 i:12,pairs changed 4
fullSet,iter: 0 i:13,pairs changed 4
fullSet,iter: 0 i:14,pairs changed 4
fullSet,iter: 0 i:15,pairs changed 5
fullSet,iter: 0 i:16,pairs changed 5
L==H
fullSet,iter: 0 i:17,pairs changed 5
j not moving enough
fullSet,iter: 0 i:18,pairs changed 5
fullSet,iter: 0 i:19,pairs changed 5
fullSet,iter: 0 i:20,pairs changed 5
fullSet,iter: 0 i:21,pairs changed 5
j not moving enough
fullSet,iter: 0 i:22,pairs changed 5
L==H
fullSet,iter: 0 i:23,pairs changed 5
fullSet,iter: 0 i:24,pairs changed 5
fullSet,iter: 0

In [22]:
print(b,alphas[alphas>0])
print(np.shape(alphas[alphas>0]))

[[-3.57974049]] [[0.04916579 0.05794998 0.16680268 0.02385687 0.02385687 0.10711577
  0.16680268]]
(1, 7)


常数C给出的是不同优化问题的权重，常数C一方面要保障所有样例间隔不小于1.0，另一方面又要使得分类间隔要尽可能大，并且要在这两方面之间平衡。如果C很大，那么分类器将力图通过分隔超平面对所有样例都正确分类。如果数据集非线性可分，就会发现支持向量会在超平面附近聚集成团。 

In [25]:
#利用alpha进行分类
def calcWs(alphas,dataArr,classLabels):
    X=np.mat(dataArr)
    labelMat=np.mat(classLabels).transpose()
    m,n=np.shape(X)
    w=np.zeros((n,1))
    for i in range(m):
        w+=np.multiply(alphas[i]*labelMat[i],X[i,:].T)
    return w

In [26]:
ws=calcWs(alphas,dataArr,labelArr)
print(ws)

[[ 0.76671715]
 [-0.19656689]]


In [28]:
dataMat=np.mat(dataArr)
print(dataMat[0]*np.mat(ws)+b)
print(labelArr[0])

[[-1.25234746]]
-1.0


### 在复杂数据上应用核函数

In [52]:
#核转换函数
def kernelTrans(X, A, kTup): #calc the kernel or transform data to a higher dimensional space
    m,n = np.shape(X)
    K = np.mat(np.zeros((m,1)))
    if kTup[0]=='lin': K = X * A.T   #linear kernel
    elif kTup[0]=='rbf':
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow*deltaRow.T
        K = np.exp(K/(-1*kTup[1]**2)) #divide in NumPy is element-wise not matrix like Matlab
    else: raise NameError('Houston We Have a Problem -- That Kernel is not recognized')
    return K

class optStruct:
    def __init__(self,dataMatIn,classLabels,C,toler,kTup):
        self.X=dataMatIn
        self.labelMat=classLabels
        self.C=C
        self.tol=toler
        self.m=np.shape(dataMatIn)[0]
        self.alphas=np.mat(np.zeros((self.m,1)))
        self.b=0
        self.eCache=np.mat(np.zeros((self.m,2)))
        self.K=np.mat(np.zeros((self.m,self.m)))  #############################
        for i in range(self.m): ###########################
            self.K[:,i]=kernelTrans(self.X,self.X[i,:],kTup) #######################

#使用核函数，需要对innerL和calcEk函数进行修改
def innerL(i,oS):
    Ei=calcEk(oS,i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i]>0)):
        j,Ej=selectJ(i,oS,Ei)
        alphaIold=oS.alphas[i].copy()
        alphaJold=oS.alphas[j].copy()
        if (oS.labelMat[i] != oS.labelMat[j]):
            L=max(0,oS.alphas[j]-oS.alphas[i])
            H=min(oS.C,oS.C+oS.alphas[j]-oS.alphas[i])
        else:
            L=max(0,oS.alphas[j]+oS.alphas[i]-oS.C)
            H=min(oS.C,oS.alphas[j]+oS.alphas[i])
        if L==H:
            print("L==H")
            return 0
        eta=2.0*oS.K[i,j]-oS.K[i,i]-oS.K[j,j]   ######################修改
        if eta>=0:
            print("eta>=0")
            return 0
        oS.alphas[j]-=oS.labelMat[j]*(Ei-Ej)/eta
        oS.alphas[j]=clipAlpha(oS.alphas[j],H,L)
        updateEk(oS,j)
        if (abs(oS.alphas[j]-alphaJold)<0.00001):
            print("j not moving enough")
            return 0
        oS.alphas[i]+=oS.labelMat[j]*oS.labelMat[i]*(alphaJold-oS.alphas[j])
        updateEk(oS,i)
        b1=oS.b-Ei-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i]-\
            oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]       ##########################b1,b2修改
        b2=oS.b-Ej-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]-\
            oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if (0<oS.alphas[i]) and (oS.C>oS.alphas[j]):
            oS.b=b1
        elif (0<oS.alphas[j]) and (oS.C>oS.alphas[j]):
            oS.b=b2
        else:
            oS.b=(b1+b2)/2.0
        return 1
    else:
        return 0
def calcEk(oS,k):
    fXk=float(np.multiply(oS.alphas,oS.labelMat).T*oS.K[:,k]+oS.b)   ######################修改
    Ek=fXk-float(oS.labelMat[k])
    return Ek

In [64]:
#利用核函数进行分类的径向基测试函数
def sign(pre):
    if pre>0:
        pre=1.0
    else:
        pre=-1.0
    
def testRbf(k1=1.3):
    dataArr,labelArr = loadDataSet('testSetRBF.txt')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, ('rbf', k1)) #C=200 important
    datMat=np.mat(dataArr); labelMat = np.mat(labelArr).transpose()
    svInd=np.nonzero(alphas.A>0)[0]
    sVs=datMat[svInd] #get matrix of only support vectors
    labelSV = labelMat[svInd];
    print ("there are %d Support Vectors" % np.shape(sVs)[0])
    m,n = np.shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf', k1))
        predict=kernelEval.T * np.multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print("the training error rate is: %f" % (float(errorCount)/m))
    dataArr,labelArr = loadDataSet('testSetRBF2.txt')
    errorCount = 0
    datMat=np.mat(dataArr); labelMat = np.mat(labelArr).transpose()
    m,n = np.shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf', k1))
        predict=kernelEval.T * np.multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1    
    print( "the test error rate is: %f" % (float(errorCount)/m) )

In [65]:
testRbf()

fullSet,iter: 0 i:0,pairs changed 1
fullSet,iter: 0 i:1,pairs changed 1
fullSet,iter: 0 i:2,pairs changed 2
fullSet,iter: 0 i:3,pairs changed 3
fullSet,iter: 0 i:4,pairs changed 3
fullSet,iter: 0 i:5,pairs changed 4
fullSet,iter: 0 i:6,pairs changed 4
fullSet,iter: 0 i:7,pairs changed 5
fullSet,iter: 0 i:8,pairs changed 6
fullSet,iter: 0 i:9,pairs changed 6
fullSet,iter: 0 i:10,pairs changed 6
fullSet,iter: 0 i:11,pairs changed 7
fullSet,iter: 0 i:12,pairs changed 7
fullSet,iter: 0 i:13,pairs changed 8
fullSet,iter: 0 i:14,pairs changed 9
fullSet,iter: 0 i:15,pairs changed 10
fullSet,iter: 0 i:16,pairs changed 11
fullSet,iter: 0 i:17,pairs changed 12
fullSet,iter: 0 i:18,pairs changed 13
fullSet,iter: 0 i:19,pairs changed 14
fullSet,iter: 0 i:20,pairs changed 14
fullSet,iter: 0 i:21,pairs changed 15
fullSet,iter: 0 i:22,pairs changed 15
fullSet,iter: 0 i:23,pairs changed 15
fullSet,iter: 0 i:24,pairs changed 16
j not moving enough
fullSet,iter: 0 i:25,pairs changed 16
fullSet,iter: 0 i

j not moving enough
fullSet,iter: 8 i:13,pairs changed 0
j not moving enough
fullSet,iter: 8 i:14,pairs changed 0
j not moving enough
fullSet,iter: 8 i:15,pairs changed 0
j not moving enough
fullSet,iter: 8 i:16,pairs changed 0
j not moving enough
fullSet,iter: 8 i:17,pairs changed 0
j not moving enough
fullSet,iter: 8 i:18,pairs changed 0
fullSet,iter: 8 i:19,pairs changed 0
fullSet,iter: 8 i:20,pairs changed 0
j not moving enough
fullSet,iter: 8 i:21,pairs changed 0
fullSet,iter: 8 i:22,pairs changed 0
j not moving enough
fullSet,iter: 8 i:23,pairs changed 0
fullSet,iter: 8 i:24,pairs changed 0
fullSet,iter: 8 i:25,pairs changed 0
j not moving enough
fullSet,iter: 8 i:26,pairs changed 0
j not moving enough
fullSet,iter: 8 i:27,pairs changed 0
fullSet,iter: 8 i:28,pairs changed 0
j not moving enough
fullSet,iter: 8 i:29,pairs changed 0
L==H
fullSet,iter: 8 i:30,pairs changed 0
fullSet,iter: 8 i:31,pairs changed 0
fullSet,iter: 8 i:32,pairs changed 0
j not moving enough
fullSet,iter: 8

### 示例：手写识别问题

In [82]:
def img2vector(filename):
    returnVect = np.zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect

def loadImages(dirName):
    from os import listdir
    hwLabels = []
    trainingFileList = listdir(dirName)           #load the training set
    m = len(trainingFileList)
    trainingMat = np.zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]     #take off .txt
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9: hwLabels.append(-1)
        else: hwLabels.append(1)
        trainingMat[i,:] = img2vector('%s/%s' % (dirName, fileNameStr))
    return trainingMat, hwLabels    

def testDigits(kTup=('rbf', 10)):
    dataArr,labelArr = loadImages('digits/trainingDigits')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, kTup)
    datMat=np.mat(dataArr); labelMat = np.mat(labelArr).transpose()
    svInd=np.nonzero(alphas.A>0)[0]
    sVs=datMat[svInd] 
    labelSV = labelMat[svInd];
    print("there are %d Support Vectors" % np.shape(sVs)[0])
    m,n = np.shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * np.multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print("the training error rate is: %f" % (float(errorCount)/m))
    dataArr,labelArr = loadImages('digits/testDigits')
    errorCount = 0
    datMat=np.mat(dataArr); labelMat = np.mat(labelArr).transpose()
    m,n = np.shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * np.multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1    
    print("the test error rate is: %f" % (float(errorCount)/m) )


In [83]:
testDigits(('rbf',20))

fullSet,iter: 0 i:0,pairs changed 1
fullSet,iter: 0 i:1,pairs changed 2
fullSet,iter: 0 i:2,pairs changed 3
fullSet,iter: 0 i:3,pairs changed 4
fullSet,iter: 0 i:4,pairs changed 5
fullSet,iter: 0 i:5,pairs changed 6
fullSet,iter: 0 i:6,pairs changed 6
fullSet,iter: 0 i:7,pairs changed 7
fullSet,iter: 0 i:8,pairs changed 8
fullSet,iter: 0 i:9,pairs changed 9
fullSet,iter: 0 i:10,pairs changed 10
fullSet,iter: 0 i:11,pairs changed 10
fullSet,iter: 0 i:12,pairs changed 11
fullSet,iter: 0 i:13,pairs changed 12
fullSet,iter: 0 i:14,pairs changed 13
fullSet,iter: 0 i:15,pairs changed 13
fullSet,iter: 0 i:16,pairs changed 13
fullSet,iter: 0 i:17,pairs changed 13
fullSet,iter: 0 i:18,pairs changed 14
fullSet,iter: 0 i:19,pairs changed 14
fullSet,iter: 0 i:20,pairs changed 15
L==H
fullSet,iter: 0 i:21,pairs changed 15
fullSet,iter: 0 i:22,pairs changed 15
L==H
fullSet,iter: 0 i:23,pairs changed 15
fullSet,iter: 0 i:24,pairs changed 15
fullSet,iter: 0 i:25,pairs changed 15
fullSet,iter: 0 i:26,p

L==H
fullSet,iter: 0 i:278,pairs changed 31
j not moving enough
fullSet,iter: 0 i:279,pairs changed 31
fullSet,iter: 0 i:280,pairs changed 31
L==H
fullSet,iter: 0 i:281,pairs changed 31
L==H
fullSet,iter: 0 i:282,pairs changed 31
fullSet,iter: 0 i:283,pairs changed 31
L==H
fullSet,iter: 0 i:284,pairs changed 31
fullSet,iter: 0 i:285,pairs changed 31
fullSet,iter: 0 i:286,pairs changed 31
L==H
fullSet,iter: 0 i:287,pairs changed 31
fullSet,iter: 0 i:288,pairs changed 31
L==H
fullSet,iter: 0 i:289,pairs changed 31
L==H
fullSet,iter: 0 i:290,pairs changed 31
L==H
fullSet,iter: 0 i:291,pairs changed 31
L==H
fullSet,iter: 0 i:292,pairs changed 31
L==H
fullSet,iter: 0 i:293,pairs changed 31
fullSet,iter: 0 i:294,pairs changed 31
fullSet,iter: 0 i:295,pairs changed 31
L==H
fullSet,iter: 0 i:296,pairs changed 31
L==H
fullSet,iter: 0 i:297,pairs changed 31
L==H
fullSet,iter: 0 i:298,pairs changed 31
L==H
fullSet,iter: 0 i:299,pairs changed 31
fullSet,iter: 0 i:300,pairs changed 31
L==H
fullSet,

j not moving enough
non-bound,iter:3 i 207,pairs changed 2
j not moving enough
non-bound,iter:3 i 208,pairs changed 2
j not moving enough
non-bound,iter:3 i 215,pairs changed 2
j not moving enough
non-bound,iter:3 i 235,pairs changed 2
j not moving enough
non-bound,iter:3 i 249,pairs changed 2
j not moving enough
non-bound,iter:3 i 250,pairs changed 2
j not moving enough
non-bound,iter:3 i 254,pairs changed 2
j not moving enough
non-bound,iter:3 i 259,pairs changed 2
j not moving enough
non-bound,iter:3 i 284,pairs changed 2
j not moving enough
non-bound,iter:3 i 287,pairs changed 2
j not moving enough
non-bound,iter:3 i 293,pairs changed 2
j not moving enough
non-bound,iter:3 i 297,pairs changed 2
j not moving enough
non-bound,iter:3 i 305,pairs changed 2
j not moving enough
non-bound,iter:3 i 306,pairs changed 2
j not moving enough
non-bound,iter:3 i 311,pairs changed 2
j not moving enough
non-bound,iter:3 i 317,pairs changed 2
j not moving enough
non-bound,iter:3 i 319,pairs changed

j not moving enough
fullSet,iter: 5 i:101,pairs changed 0
fullSet,iter: 5 i:102,pairs changed 0
fullSet,iter: 5 i:103,pairs changed 0
fullSet,iter: 5 i:104,pairs changed 0
j not moving enough
fullSet,iter: 5 i:105,pairs changed 0
fullSet,iter: 5 i:106,pairs changed 0
fullSet,iter: 5 i:107,pairs changed 0
fullSet,iter: 5 i:108,pairs changed 0
fullSet,iter: 5 i:109,pairs changed 0
j not moving enough
fullSet,iter: 5 i:110,pairs changed 0
j not moving enough
fullSet,iter: 5 i:111,pairs changed 0
j not moving enough
fullSet,iter: 5 i:112,pairs changed 0
L==H
fullSet,iter: 5 i:113,pairs changed 0
fullSet,iter: 5 i:114,pairs changed 0
L==H
fullSet,iter: 5 i:115,pairs changed 0
j not moving enough
fullSet,iter: 5 i:116,pairs changed 0
L==H
fullSet,iter: 5 i:117,pairs changed 0
fullSet,iter: 5 i:118,pairs changed 0
j not moving enough
fullSet,iter: 5 i:119,pairs changed 0
L==H
fullSet,iter: 5 i:120,pairs changed 0
fullSet,iter: 5 i:121,pairs changed 0
fullSet,iter: 5 i:122,pairs changed 0
L==H

L==H
fullSet,iter: 5 i:302,pairs changed 1
L==H
fullSet,iter: 5 i:303,pairs changed 1
L==H
fullSet,iter: 5 i:304,pairs changed 1
j not moving enough
fullSet,iter: 5 i:305,pairs changed 1
j not moving enough
fullSet,iter: 5 i:306,pairs changed 1
fullSet,iter: 5 i:307,pairs changed 1
fullSet,iter: 5 i:308,pairs changed 1
fullSet,iter: 5 i:309,pairs changed 1
L==H
fullSet,iter: 5 i:310,pairs changed 1
j not moving enough
fullSet,iter: 5 i:311,pairs changed 1
j not moving enough
fullSet,iter: 5 i:312,pairs changed 1
fullSet,iter: 5 i:313,pairs changed 1
fullSet,iter: 5 i:314,pairs changed 1
L==H
fullSet,iter: 5 i:315,pairs changed 1
L==H
fullSet,iter: 5 i:316,pairs changed 1
j not moving enough
fullSet,iter: 5 i:317,pairs changed 1
L==H
fullSet,iter: 5 i:318,pairs changed 1
j not moving enough
fullSet,iter: 5 i:319,pairs changed 1
j not moving enough
fullSet,iter: 5 i:320,pairs changed 1
L==H
fullSet,iter: 5 i:321,pairs changed 1
j not moving enough
fullSet,iter: 5 i:322,pairs changed 1
j 

fullSet,iter: 7 i:71,pairs changed 0
fullSet,iter: 7 i:72,pairs changed 0
fullSet,iter: 7 i:73,pairs changed 0
fullSet,iter: 7 i:74,pairs changed 0
fullSet,iter: 7 i:75,pairs changed 0
fullSet,iter: 7 i:76,pairs changed 0
fullSet,iter: 7 i:77,pairs changed 0
fullSet,iter: 7 i:78,pairs changed 0
fullSet,iter: 7 i:79,pairs changed 0
fullSet,iter: 7 i:80,pairs changed 0
fullSet,iter: 7 i:81,pairs changed 0
fullSet,iter: 7 i:82,pairs changed 0
fullSet,iter: 7 i:83,pairs changed 0
fullSet,iter: 7 i:84,pairs changed 0
j not moving enough
fullSet,iter: 7 i:85,pairs changed 0
fullSet,iter: 7 i:86,pairs changed 0
j not moving enough
fullSet,iter: 7 i:87,pairs changed 0
fullSet,iter: 7 i:88,pairs changed 0
fullSet,iter: 7 i:89,pairs changed 0
fullSet,iter: 7 i:90,pairs changed 0
fullSet,iter: 7 i:91,pairs changed 0
fullSet,iter: 7 i:92,pairs changed 0
fullSet,iter: 7 i:93,pairs changed 0
fullSet,iter: 7 i:94,pairs changed 0
fullSet,iter: 7 i:95,pairs changed 0
fullSet,iter: 7 i:96,pairs changed 

L==H
fullSet,iter: 7 i:275,pairs changed 0
j not moving enough
fullSet,iter: 7 i:276,pairs changed 0
L==H
fullSet,iter: 7 i:277,pairs changed 0
L==H
fullSet,iter: 7 i:278,pairs changed 0
L==H
fullSet,iter: 7 i:279,pairs changed 0
fullSet,iter: 7 i:280,pairs changed 0
j not moving enough
fullSet,iter: 7 i:281,pairs changed 0
L==H
fullSet,iter: 7 i:282,pairs changed 0
j not moving enough
fullSet,iter: 7 i:283,pairs changed 0
j not moving enough
fullSet,iter: 7 i:284,pairs changed 0
fullSet,iter: 7 i:285,pairs changed 0
fullSet,iter: 7 i:286,pairs changed 0
j not moving enough
fullSet,iter: 7 i:287,pairs changed 0
j not moving enough
fullSet,iter: 7 i:288,pairs changed 0
L==H
fullSet,iter: 7 i:289,pairs changed 0
L==H
fullSet,iter: 7 i:290,pairs changed 0
j not moving enough
fullSet,iter: 7 i:291,pairs changed 0
j not moving enough
fullSet,iter: 7 i:292,pairs changed 0
j not moving enough
fullSet,iter: 7 i:293,pairs changed 0
fullSet,iter: 7 i:294,pairs changed 0
fullSet,iter: 7 i:295,pai

# 这一节需要重点理解一下，最近事情多，SVM有点囫囵吞枣式学习  
还有欠缺的是PLA数据怎么跟